In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz


In [2]:
scrap_cuadros_completo = pd.read_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/scrap_cuadros_completo.csv")

In [3]:
dim_original = scrap_cuadros_completo.shape # De esta forma guardo las dimensiones del data frame original.
scrap_cuadros_completo.shape

(44724, 10)

In [4]:
cuadros_completo = scrap_cuadros_completo.copy()

# Estudio de la información de iconografía en el Museo Del Prado.

# 1. Exploración inicial.

In [5]:
cuadros_completo.head()

,Titulo,Soporte_Fecha,Autor,Personajes,Objetos,Flora,Fauna,Geografico,Escuela,Ubicacion
0,El sumo sacerdote Aarón,Óleo sobre tabla. 1545 - 1550,"JUANES, JUAN DE",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 051\nEliminar
1,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
2,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Moisés\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
3,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Abacuc\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar
4,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Amós\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar


In [6]:
cuadros_completo.columns # Esto nos proporciona una enumeración de los nombres correspondientes a cada una de las columnas presentes en los datos.

Index(['Titulo', 'Soporte_Fecha', 'Autor', 'Personajes', 'Objetos', 'Flora',
       'Fauna', 'Geografico', 'Escuela', 'Ubicacion'],
      dtype='object')

In [7]:
# Aplicar expresión regular para extraer información
cuadros_completo[['Soporte', 'Fecha']] = cuadros_completo['Soporte_Fecha'].str.extract(r'(.*?)(?:\. )?(.*)')



In [8]:
#cuadros_completo[['Soporte', 'Fecha']] = cuadros_completo['Soporte_Fecha'].str.split('.', n=1, expand=True)

In [9]:
cuadros_completo.drop('Soporte_Fecha', axis=1, inplace=True)

In [10]:
cuadros_completo.info() # De este modo conzco el tipo de dato y los valores nulos que tiene cada una de las columnas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44724 entries, 0 to 44723
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Titulo      44724 non-null  object
 1   Autor       44687 non-null  object
 2   Personajes  13157 non-null  object
 3   Objetos     30494 non-null  object
 4   Flora       27784 non-null  object
 5   Fauna       30674 non-null  object
 6   Geografico  4508 non-null   object
 7   Escuela     12029 non-null  object
 8   Ubicacion   33262 non-null  object
 9   Soporte     44724 non-null  object
 10  Fecha       44724 non-null  object
dtypes: object(11)
memory usage: 3.8+ MB


In [6]:
cuadros_completo['Soporte_Fecha'].unique()

array(['Óleo sobre tabla. 1545 - 1550', 'Óleo sobre lienzo. 1566 - 1568',
       'Temple sobre tabla. 1435 - 1440', ...,
       'Óleo sobre lienzo. 1624 - 1662', 'Óleo sobre lienzo. Hacia 1763',
       'Óleo sobre lienzo. Hacia 1769'], dtype=object)

In [11]:
cuadros_completo['Soporte'].unique()

array([''], dtype=object)

In [8]:
# Renombrar las columnas
obras_completo = obras_completo.rename(columns={
    'uriObra': 'Obra_uri',
    'tituloObra': 'Titulo',
    'autores': 'Autor',
    'fecha': 'Fecha',
    'anio': 'Año',
    'Floras': 'Flora',
    'Escuelas': 'Escuela',
    'Faunas': 'Fauna',
    'LugaresGeo': 'Lugar',
})

In [10]:
cuadros_completo.head()

,Titulo,Soporte_Fecha,Autor,Personajes,Objetos,Flora,Fauna,Geografico,Escuela,Ubicacion
0,El sumo sacerdote Aarón,Óleo sobre tabla. 1545 - 1550,"JUANES, JUAN DE",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 051\nEliminar
1,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Aarón\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
2,Los israelitas bebiendo el agua milagrosa,Óleo sobre lienzo. 1566 - 1568,"BASSANO, JACOPO",Moisés\nEliminar,NaN,NaN,NaN,NaN,NaN,NaN
3,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Abacuc\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar
4,Retablo de la Virgen,Temple sobre tabla. 1435 - 1440,MAESTRO DE TORRALBA,Amós\nEliminar,NaN,NaN,NaN,NaN,NaN,Sala 052A\nEliminar


# Análisis de "Fecha" y "Año"

In [11]:
obras_completo[['Fecha', 'Año']]

,Fecha,Año
0,1640 - 1642,1640
1,Hacia 1500,1495
2,1570 - 1600,1570
3,Hacia 1676,1671
4,Hacia 1781,1776
...,...,...
650,Antes de 1649,1629
651,1876,1876
652,Hacia 1520,1515
653,Primera mitad del siglo XVII,1601


In [12]:
def transformar_años(año):
    if año == -10:
        return -100
    elif año == -20:
        return -200
    elif año == -30:
        return -300
    elif año == -40:
        return -400
    else:
        return año
      

obras_completo['Año'] = obras_completo['Año'].apply(transformar_años)

In [13]:
obras_completo['Año'].unique()

array([1640, 1495, 1570, 1671, 1776, 1601, 1924, 1856, 1727, 1615, 1663,
       1505, 1642, 1908, 1832, 1660, 1689, 1550, 1874, 1650, 1685,  -21,
       1787, 1635, 1859, 1501, 1515, 1545, 1906, 1490, 1885, 1585, 1659,
       1867, 1551, 1799, 1805, 1826, 1425, 1829, 1595, 1783, 1654, 1780,
       1847, 1670, 1901, 1919, 1628, 1779, 1620, 1751, 1872, 1846, 1634,
       1769, 1633, 1857, 1643, 1630, 1655, 1913, 1804, 1631, 1600, 1868,
       1870, 1915, 1603, 1900, 1811, 1541, 1910, 1658, 1795, 1692, 1627,
       1651, 1803, 1724, 1875, 1423, 1624, 1871, 1869, 1888, 1675, 1695,
       1680, 1690, 1719, 1567,   50, 1701, 1770, 1890, 1790, 1876, 1694,
       1610, 1785, 1898, 1914, 1858, 1882, 1881, 1844, 1616, 1806, 1668,
       1612, 1784, -100, 1866, 1877, 1879, 1862, 1678, 1676, 1617, 1887,
       1813, 1638, 1531, 1491, 1664, 1745, 1781, -300, 1892, 1665, 1606,
       1775, 1465, 1644, 1604, 1860, 1509, 1895, 1789, 1764, 1760, 1884,
       1837, 1774, 1480, 1801, 1969, 1645, 1842, 18

In [14]:
#Puede verse que un dato cuerioso es el -21, vamos a ver que ocurre ahí:

In [15]:
obras_completo.loc[obras_completo['Año'] == -21]

,Obra_uri,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imagenes
28,http://museodelprado.es/items/E22_Man-Made_Obj...,Hércules / Palmera con frutos,Anonymous ; Anónimo,Finales del siglo III a.C. - Primera mitad del...,-21,Hispano-cartaginesa,NaN,NaN,Palmera (Phoenix dactylifera),NaN,Hércules/Heracles,https://content3.cdnprado.net/imagenes/Documen...


In [16]:
#en efecto, coincide con el siglo III a.C, por lo que lo modificaremos por un -210.

In [17]:
obras_completo['Año'] = obras_completo['Año'].replace(-21, -210)

In [18]:
obras_completo['Año'].unique()

array([1640, 1495, 1570, 1671, 1776, 1601, 1924, 1856, 1727, 1615, 1663,
       1505, 1642, 1908, 1832, 1660, 1689, 1550, 1874, 1650, 1685, -210,
       1787, 1635, 1859, 1501, 1515, 1545, 1906, 1490, 1885, 1585, 1659,
       1867, 1551, 1799, 1805, 1826, 1425, 1829, 1595, 1783, 1654, 1780,
       1847, 1670, 1901, 1919, 1628, 1779, 1620, 1751, 1872, 1846, 1634,
       1769, 1633, 1857, 1643, 1630, 1655, 1913, 1804, 1631, 1600, 1868,
       1870, 1915, 1603, 1900, 1811, 1541, 1910, 1658, 1795, 1692, 1627,
       1651, 1803, 1724, 1875, 1423, 1624, 1871, 1869, 1888, 1675, 1695,
       1680, 1690, 1719, 1567,   50, 1701, 1770, 1890, 1790, 1876, 1694,
       1610, 1785, 1898, 1914, 1858, 1882, 1881, 1844, 1616, 1806, 1668,
       1612, 1784, -100, 1866, 1877, 1879, 1862, 1678, 1676, 1617, 1887,
       1813, 1638, 1531, 1491, 1664, 1745, 1781, -300, 1892, 1665, 1606,
       1775, 1465, 1644, 1604, 1860, 1509, 1895, 1789, 1764, 1760, 1884,
       1837, 1774, 1480, 1801, 1969, 1645, 1842, 18

In [19]:
def transformar_fechas(fecha):
    dic_fechas = {
        'Primer cuarto del siglo XVII': '1601 - 1625',
        'Siglo XVII': '1601 - 1700',
        'Finales del siglo XVII - Principio del siglo XVIII': '1685 - 1715',
        'Finales del siglo III a.C. - Primera mitad del siglo II a.C.': '-30 - -25',
        'Siglo XVI': '1501 - 1600',
        'Segunda mitad del siglo XVI': '1551 - 1600',
        'Principio del siglo XX': '1901 - 1915',
        'Último tercio del siglo XVIII': '1769 - 1800',
        'Finales del siglo XVI - Primer tercio del siglo XVII': '1585 - 1630',
        'Primera mitad del siglo XVII': '1601 - 1650',
        'Segunda mitad del siglo XVII': '1651 - 1700',
        'Finales del siglo XVII': '1685 - 1700',
        'Siglo XVIII': '1701 - 1800',
        'Finales del siglo XVIII': '1785 - 1800',
        'Siglos XVI - XVII': '1501 - 1700',
        'Segunda mitad del siglo XVII - Primer cuarto del siglo XVIII': '1651 - 1725',
        'Siglo I a.C.': '-100',
        'Finales del siglo XVI': '1685 - 1700',
        'Primer tercio del siglo XVII': '1601 - 1630',
        'Siglo III a.C.': '-300',
        'Primer tercio del siglo XVI': '1501 - 1530',
        'Segundo tercio del siglo XVII': '1634 - 1870',
        'Siglo XII': '1101 - 1199',
        'Último cuarto del siglo XVIII - Principio del siglo XIX': '1776 - 1815',
        'Último cuarto del siglo XVII': '1676 - 1700',
        'Siglos IV a.C. - III a.C.': '-400 - -300',
        'Mediados del siglo XVII': '1640 - 1660',
        'Siglo XIX': '1801 - 1900',
    }

    return dic_fechas.get(fecha, fecha)

obras_completo['rango_fecha'] = obras_completo['Fecha'].apply(transformar_fechas)


In [20]:
# Si 'Hacia' está presente en la columna 'Fecha', tomar el valor de la columna 'Año'; sino, mantener el valor de la columna 'Fecha'

def asignar_rango_fecha(celda):
    if 'Hacia' in celda['Fecha']:
        return celda['Año']
    else:
        return celda['Fecha']

# Aplicar la función a cada celda del DataFrame y almacenar el resultado en la nueva columna 'Fecha_rango'
obras_completo['rango_fecha'] = obras_completo.apply(asignar_rango_fecha, axis=1)


# Análisis "Autor"

In [21]:
obras_completo.head()

,Obra_uri,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imagenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla ; Master of La Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


In [22]:
nombres_unicos = obras_completo['Autor'].unique()
nombres_unicos = list(nombres_unicos)
nombres_unicos

['Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)',
 'Maestro de la Sisla ; Master of La Sisla',
 'Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi',
 'Pérez, Bartolomé',
 'Maella, Mariano Salvador',
 "Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
 'Martí Garcés de Marcilla, José',
 'Haes, Carlos de',
 'Tobar, Alonso Miguel de',
 'Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan',
 'Camprobín, Pedro de',
 'Rincón, Fernando del',
 'Hiepes, Tomás',
 'Bertodano, Luis de',
 'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid',
 'Vauquer, Jacques',
 'Coosemans, Alexander',
 'Leoni, Leone',
 'J. Laurent y Cía',
 'Arellano, Juan de',
 'Galleria dei Lavori, Florencia',
 'Belvedere, Andrea ; Solimena, Francesco',
 'Anonymous ; Anónimo',
 'Malaine, Joseph-Laurent',
 'Dyck, Anthony van ; Dyck, Anton van',
 'Puebla y Tolín, Dióscoro Teófilo',
 'Maestro de la Santa Sangre',
 'David, Gérard'

In [23]:
for e in nombres_unicos:
    print(e)

Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)
Maestro de la Sisla ; Master of La Sisla
Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi
Pérez, Bartolomé
Maella, Mariano Salvador
Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)
Martí Garcés de Marcilla, José
Haes, Carlos de
Tobar, Alonso Miguel de
Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan
Camprobín, Pedro de
Rincón, Fernando del
Hiepes, Tomás
Bertodano, Luis de
Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid
Vauquer, Jacques
Coosemans, Alexander
Leoni, Leone
J. Laurent y Cía
Arellano, Juan de
Galleria dei Lavori, Florencia
Belvedere, Andrea ; Solimena, Francesco
Anonymous ; Anónimo
Malaine, Joseph-Laurent
Dyck, Anthony van ; Dyck, Anton van
Puebla y Tolín, Dióscoro Teófilo
Maestro de la Santa Sangre
David, Gérard
Titian (Tiziano Vecellio) ; Tiziano, Vecellio di Gregorio
Pulido Fernández, Ramón
Pereda y Salgado, Antonio de

In [24]:
nombres_unicos[0]

'Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)'

In [25]:
descomposicion_de_nombres = nombres_unicos[0].split(";")
descomposicion_de_nombres

['Nuzzi, Mario ', ' Nuzzi, Mario (Mario dei Fiori)']

In [26]:
for i in descomposicion_de_nombres:
    print(i)

Nuzzi, Mario 
 Nuzzi, Mario (Mario dei Fiori)


In [27]:
def elegir_autor(nombres):
    # Divide la cadena de nombres en una lista y elimina los espacios alrededor de cada nombre
    lista_nombres = [nombre.strip() for nombre in nombres.split(';')]
    
    # Calcula la similitud entre los conjuntos de palabras
    puntajes_similitud = []
    for i in range(len(lista_nombres)):
        for j in range(i + 1, len(lista_nombres)):
            puntaje = fuzz.token_set_ratio(lista_nombres[i], lista_nombres[j])
            puntajes_similitud.append((i, j, puntaje))
    
    # Ordena la lista de similitudes de mayor a menor
    puntajes_similitud.sort(key=lambda x: x[2], reverse=True)
    
    # Devuelve el nombre con mayor similitud si supera el umbral del 60%
    if puntajes_similitud and puntajes_similitud[0][2] > 60:  # Puedes ajustar el umbral según tus necesidades
        return lista_nombres[puntajes_similitud[0][0]]
    else:
        return nombres

In [28]:
obras_completo['Autor'] = obras_completo['Autor'].apply(elegir_autor)

In [29]:
obras_completo.head()

,Obra_uri,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imagenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


In [30]:
# Filtrar las filas donde la columna 'Autor' contiene un punto y coma
autor_puntocoma = obras_completo[obras_completo['Autor'].str.contains(';')]['Autor']

# Obtener una lista de valores únicos
print(autor_puntocoma.unique())

['Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi'
 'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Belvedere, Andrea ; Solimena, Francesco'
 'Blanco y Assensio, Alejandro ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Legrand, Luis Carlos ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Decraene, Florentino ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'López, Juan Antonio ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid'
 'Quellinus, Erasmus ; Seghers, Daniel' 'Bosch, Hieronymus ; El Bosco'
 'Belvedere, Andrea ; Giordano, Luca'
 'Flémalle, Bertholet ; Legrand, Luis Carlos ; Madrazo y Agudo, José de ; Poussin, Nicolas ; Real Establecimiento Litográfico de Madrid'
 'Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid ; Rodríguez, Cayetano'
 'Decraene, Florentino ; Madrazo y Agudo, José de ; Murillo,

In [31]:
# Reemplazar 'Anonymous' por 'Anonimo' en la columna 'Autor'
obras_completo['Autor'] = obras_completo['Autor'].replace('Anonymous', 'Anonimo')


# Análisis "Escuela"

In [32]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala', nan,
       'Púnica', 'Hispano-flamenca', 'Española ; Valenciana',
       'Florentina ; Italiana', 'Inglesa', 'Sueca',
       'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [33]:
obras_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Obra_uri     655 non-null    object
 1   Titulo       655 non-null    object
 2   Autor        655 non-null    object
 3   Fecha        655 non-null    object
 4   Año          655 non-null    int64 
 5   Escuela      649 non-null    object
 6   Objetos      319 non-null    object
 7   Fauna        188 non-null    object
 8   Flora        655 non-null    object
 9   Lugar        48 non-null     object
 10  Personajes   309 non-null    object
 11  Imagenes     655 non-null    object
 12  rango_fecha  655 non-null    object
dtypes: int64(1), object(12)
memory usage: 66.7+ KB


In [34]:
obras_completo['Escuela'].isna().any()

True

In [35]:
obras_completo['Escuela'] = obras_completo['Escuela'].fillna('Desconocida')

In [36]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala',
       'Desconocida', 'Púnica', 'Hispano-flamenca',
       'Española ; Valenciana', 'Florentina ; Italiana', 'Inglesa',
       'Sueca', 'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [37]:
#En el análisis subsiguiente, no tiene sentido alguno hacer un estudio en el que se comparen la escuela toledana, por ejemplo, con la española, pues una implica a la otra, por este motivo, dejaremos, al igual que en Lugares, unicamente las escuelas mas generales.

In [38]:
def modificar_escuela(escuela):
    palabras_clave = ['Española', 'Italiana', 'Francesa']
    
    # Verifica si cada palabra clave está en la cadena y devuelve la primera encontrada
    for palabra in palabras_clave:
        if palabra in escuela:
            return palabra
    
    # Si no se encuentra ninguna palabra clave, devuelve la cadena original
    return escuela

# Aplicar la función a la columna 'Escuela'
obras_completo['Escuela'] = obras_completo['Escuela'].apply(modificar_escuela)


In [39]:
obras_completo['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Holandesa', 'Alemana',
       'Arte Clásico ; Roma', 'Gala', 'Desconocida', 'Púnica',
       'Hispano-flamenca', 'Inglesa', 'Sueca', 'Portuguesa'], dtype=object)

In [40]:
# Modificar la columna 'Escuela' después de obtener los resultados anteriores
obras_completo['Escuela'] = obras_completo['Escuela'].replace('Arte Clásico ; Roma', 'Arte Clásico')


In [41]:
obras_completo.head()

,Obra_uri,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Imagenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,https://content3.cdnprado.net/imagenes/Documen...,1495
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,https://content3.cdnprado.net/imagenes/Documen...,1671
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,https://content3.cdnprado.net/imagenes/Documen...,1776


In [42]:
nuevo_orden = ["Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Fauna", "Flora", "Lugar", "Personajes", "Obra_uri", "Imagenes", "rango_fecha"]
obras_completo = obras_completo.reindex(columns=nuevo_orden)

In [43]:
obras_completo.head()

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Fauna,Flora,Lugar,Personajes,Obra_uri,Imagenes,rango_fecha
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos ; Florero ; Instrumentos de...,NaN,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,NaN,NaN,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,NaN,Azucena (Lilium candidum),NaN,Virgen María,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1495
2,Copa de la Vendimia,"Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Italiana,NaN,NaN,Vid / Parra (Vitis vinifera),NaN,NaN,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Española,NaN,Artrópodos (Filo: Arthropoda),Ajenuz / Arañuela (Nigella damascena) ; Anémon...,NaN,NaN,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1671
4,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Española,Elementos heráldicos y nobiliarios ; Insignia ...,NaN,Azucena (Lilium candidum),NaN,Virgen María ; Ángel / ángeles,http://museodelprado.es/items/E22_Man-Made_Obj...,https://content3.cdnprado.net/imagenes/Documen...,1776


# Análisis "Lugar"

In [44]:
#voy a hacer, por el mimso motivo que en escuela, lo mismo para lugar, no tiene sentido hacer un estudio del mismo modo que àra el resto de iconografía en el que se comparen países, ciudades, etc.

In [45]:
def modificar_lugar(lugar):
    if pd.notna(lugar):  # Verifica si no es NaN
        palabras_lugar = ['España', 'Italia', 'Francia', 'Marruecos', 'Bélgica']
        
        # Verifica si cada palabra clave está en la cadena y devuelve la primera encontrada
        for palabra in palabras_lugar:
            if palabra in lugar:
                return palabra
    
    # Si no se encuentra ninguna palabra clave o es NaN, devuelve la cadena original o NaN
    return lugar

# Aplicar la función a la columna 'Lugar'
obras_completo['Lugar'] = obras_completo['Lugar'].apply(modificar_lugar)


In [46]:
obras_completo['Lugar'].unique()

array([nan, 'España', 'Italia', 'Francia', 'Marruecos', 'Bélgica',
       'Argelia'], dtype=object)

# Creación DataFrame para análisis.

In [49]:
obras_estudio = obras_completo.copy()

In [50]:
obras = obras_estudio[["Titulo", "Autor", "Fecha", "Año", "Escuela", "rango_fecha", "Lugar"]]
objetos = obras_estudio[["Titulo", "Objetos"]]
personajes = obras_estudio[["Titulo", "Personajes"]]
flora = obras_estudio[["Titulo", "Flora"]]
fauna = obras_estudio[["Titulo", "Fauna"]]
url_imagenes_obras = obras_estudio[["Titulo", "Obra_uri", "Imagenes"]]


In [51]:
obras['Autor'] = obras['Autor'].str.split(';')
obras = obras.explode('Autor')
obras['Autor'] = obras['Autor'].str.strip()

In [52]:
personajes['Personajes'] = personajes['Personajes'].str.split(';')
personajes = personajes.explode('Personajes')
personajes['Personajes'] = personajes['Personajes'].str.strip()

In [53]:
objetos['Objetos'] = objetos['Objetos'].str.split(';')
objetos = objetos.explode('Objetos')
objetos['Objetos'] = objetos['Objetos'].str.strip()

In [54]:
flora['Flora'] = flora['Flora'].str.split(';')
flora = flora.explode('Flora')
flora['Flora'] = flora['Flora'].str.strip()

In [58]:
fauna['Fauna'] = fauna['Fauna'].str.split(';')
fauna = fauna.explode('Fauna')
fauna['Fauna'] = fauna['Fauna'].str.strip()

In [59]:
obras_estudio = pd.merge(obras, objetos, on='Titulo', how='inner')


In [60]:
obras_estudio = pd.merge(obras_estudio, personajes, on='Titulo', how='inner')

In [58]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'


In [57]:
obras_estudio = pd.merge(obras_estudio, flora, on='Titulo', how='inner')


KeyboardInterrupt: 

In [61]:
obras_estudio = pd.merge(obras_estudio, fauna, on='Titulo', how='inner')

In [62]:
nuevo_orden = ["Titulo", "Autor", "Fecha", "Año", "Escuela", "Objetos", "Personajes", "Fauna", "Lugar", "Obra_uri", "Imagenes", "rango_fecha"]
obras_estudio = obras_estudio.reindex(columns=nuevo_orden)

In [64]:
obras_estudio.head(20)

,Titulo,Autor,Fecha,Año,Escuela,Objetos,Personajes,Fauna,Lugar,Obra_uri,Imagenes,rango_fecha
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Enseres domésticos,NaN,NaN,NaN,NaN,NaN,1640 - 1642
1,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Florero,NaN,NaN,NaN,NaN,NaN,1640 - 1642
2,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Instrumentos de cuerda / cordófonos,NaN,NaN,NaN,NaN,NaN,1640 - 1642
3,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Instrumentos musicales,NaN,NaN,NaN,NaN,NaN,1640 - 1642
4,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,Violín,NaN,NaN,NaN,NaN,NaN,1640 - 1642
5,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,NaN,NaN,NaN,NaN,1495
6,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Paloma (Columba ssp),NaN,NaN,NaN,1495
7,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Paloma (Columba ssp),NaN,NaN,NaN,1495
8,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Paloma (Columba ssp),NaN,NaN,NaN,1495
9,La Anunciación,Maestro de la Sisla,Hacia 1500,1495,Española,Libro,Virgen María,Paloma (Columba ssp),NaN,NaN,NaN,1495


In [61]:
obras_estudio.shape

(50394002, 11)

# Exportación de los DataFrames

In [63]:
obras_completo.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/obras_completo.csv", index=False)

In [58]:
obras_estudio.to_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/obras_estudio.csv", index=False)

In [59]:
obras_estudio.head()

,Titulo,Autor,Fecha,Año,Escuela,rango_fecha,Lugar,Objetos,Personajes
0,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,1640 - 1642,NaN,Enseres domésticos,NaN
1,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,1640 - 1642,NaN,Florero,NaN
2,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,1640 - 1642,NaN,Instrumentos de cuerda / cordófonos,NaN
3,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,1640 - 1642,NaN,Instrumentos musicales,NaN
4,Floreros,"Nuzzi, Mario",1640 - 1642,1640,Italiana,1640 - 1642,NaN,Violín,NaN


In [2]:
obras_estudio.shape

NameError: name 'obras_estudio' is not defined